In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd


from __future__ import print_function
from typing import List, Tuple
from tqdm import tqdm
import torch

from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import DataLoader
import argparse



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (5).csv


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Load your CSV dataset
import io
df = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [ ]:


# Convert the dataset to the SQuAD format (context, question, answers)
squad_dataset = {'train': []}
for idx, row in df.iterrows():
  squad_dataset['train'].append({
         'context': row['support'],
         'question': row['question'] + ' ',
         'answers': {'text': [row['correct_answer'], row['distractor3'], row['distractor1'], row['distractor2']]}
     })

def prepare_train_features(examples):
    # Tokenize the examples using the T5 tokenizer
    tokenizer = T5Tokenizer.from_pretrained('t5-base')
    input_text = [ex['context'] + ' </s> ' + ex['question'] for ex in examples['train']]
    target_text = [ans + ' </s>' for ex in examples['train'] for ans in ex['answers']['text']]
    inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')
    outputs = tokenizer(target_text, padding=True, truncation=True, return_tensors='pt')
    return {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask'], 'decoder_input_ids': outputs['input_ids'], 'decoder_attention_mask': outputs['attention_mask'], 'labels': outputs['input_ids']}

#Define the T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-base', num_labels=4, output_attentions=False, output_hidden_states=False)

tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_accumulation_steps=50,
    save_total_limit=2,
    learning_rate=1e-4,
    warmup_steps=1000,
    adam_epsilon=1e-8,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_steps=500,
    overwrite_output_dir=True,
    save_steps=500
)

# Define the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=squad_dataset,
    data_collator=prepare_train_features,
    eval_dataset=squad_dataset
)

# Train the model
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementatio

KeyError: ignored